# Training v2.0 with a Deep Q Network (DQN) #

Import statements

In [ ]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)
%matplotlib inline

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory  # For experience replay!

In [ ]:
from gym_environment_ncml import *
from learning import *

Useful numbers

In [ ]:
MILLION = 1000000
HTHOUSAND = 100000
THOUSAND = 1000

## 1. Create environment ##

In [ ]:
env = GridworldMultiAgentv2()

In [ ]:
states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
states, actions

## 2. Create a Deep Learning Model with Keras ##

In [ ]:
model = build_model(states, actions, [32, 32, 32], ['relu', 'relu', 'relu'])

In [ ]:
model.summary()

## 3. Build Agent with Keras-RL ##

In [ ]:
dqn = build_agent(model, actions, 0.01, EpsGreedyQPolicy(), 50000)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
# dqn.compile(Adam(lr=1e-2), metrics=['mse'])

In [ ]:
history = dqn.fit(env, nb_steps=5*MILLION, visualize=False, verbose=1)

In [ ]:
history.history

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

ax.plot(history.history['nb_steps'], history.history['episode_reward'])

fig.tight_layout()
plt.show()

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Save agent to memory

In [ ]:
dqn.save_weights('agents/dqn_5b5_3030_adam_lr0.001_tmu0.01_ml50K_ns5M.h5f', overwrite=True)

## 4. Reloading Agent from memory and test ##

In [ ]:
name = 'dqn20_5b5_3216_adam_lr0.001_tmu0.01_ml50K_ns5M_eps0.1'

In [ ]:
env = GridworldMultiAgentv2(seed=2)

In [ ]:
states = env.observation_space.shape[0]
actions = env.action_space.n
model = build_model(states, actions, [32, 16], ['relu', 'relu'])
print(model.summary())
dqn = build_agent(model, actions, 0.01, EpsGreedyQPolicy(eps=0), 50000)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Load weights
dqn.load_weights(get_agent_path(name))

In [ ]:
nb_episodes = 10*THOUSAND

In [ ]:
scores = dqn.test(env, nb_episodes=nb_episodes, visualize=False, verbose=0)

In [ ]:
rewards = np.array(scores.history['episode_reward'])

In [ ]:
np.savetxt(get_test_path(name, nb_episodes), rewards)